In [77]:
CLOSE_DATASET_PATH = "/Users/valentinafeve/Datasets/13Apr"
PRODUCTS_INFO_PATH = f"{CLOSE_DATASET_PATH}/products.parquet"

In [78]:
import pandas as pd
import os

In [79]:
df_products = pd.read_parquet(PRODUCTS_INFO_PATH)
df_products.head()

,product_id,url,images,title,shop,category,category_code,material
0,133698418576796026230476316664299051166,https://us.shein.com/SHEIN-LUNE-Solid-Cold-Sho...,"b'[""https://img.ltwebstatic.com/images3_pi/202...",shein lune solid cold shoulder split sleeve po...,shein,blusa,0 1,nan
1,222071767803255529405229141031995119561,https://us.shein.com/SHEIN-LUNE-Solid-Batwing-...,"b'[""https://img.ltwebstatic.com/images3_pi/202...",shein lune solid batwing sleeve cross wrap hem...,shein,blusa,0 1,nan
2,260568335884366864206647203962898748437,https://us.shein.com/SHEIN-Essnce-Drop-Shoulde...,"b'[""https://img.ltwebstatic.com/images3_pi/202...",shein essnce drop shoulder tee biker shorts,shein,shorts,0 1,nan
3,199751947203306346644636843343713367213,https://us.shein.com/SHEIN-LUNE-Solid-Batwing-...,"b'[""https://img.ltwebstatic.com/images3_pi/202...",shein lune solid batwing sleeve hidden pocket ...,shein,vestido,0 1,nan
4,274602507127151096769612991909739354008,https://us.shein.com/Women-s-Stretch-Denim-Sho...,"b'[""https://img.ltwebstatic.com/images3_pi/202...",womens stretch denim shorts with frayed hem wa...,shein,shorts,0 1,denim


In [80]:
df_products['title'] = df_products['title'].apply(lambda x: ' '.join(x.split()[2:]))
df_products.head()

,product_id,url,images,title,shop,category,category_code,material
0,133698418576796026230476316664299051166,https://us.shein.com/SHEIN-LUNE-Solid-Cold-Sho...,"b'[""https://img.ltwebstatic.com/images3_pi/202...",solid cold shoulder split sleeve popover blouse,shein,blusa,0 1,nan
1,222071767803255529405229141031995119561,https://us.shein.com/SHEIN-LUNE-Solid-Batwing-...,"b'[""https://img.ltwebstatic.com/images3_pi/202...",solid batwing sleeve cross wrap hem tee,shein,blusa,0 1,nan
2,260568335884366864206647203962898748437,https://us.shein.com/SHEIN-Essnce-Drop-Shoulde...,"b'[""https://img.ltwebstatic.com/images3_pi/202...",drop shoulder tee biker shorts,shein,shorts,0 1,nan
3,199751947203306346644636843343713367213,https://us.shein.com/SHEIN-LUNE-Solid-Batwing-...,"b'[""https://img.ltwebstatic.com/images3_pi/202...",solid batwing sleeve hidden pocket tee dress,shein,vestido,0 1,nan
4,274602507127151096769612991909739354008,https://us.shein.com/Women-s-Stretch-Denim-Sho...,"b'[""https://img.ltwebstatic.com/images3_pi/202...",denim shorts with frayed hem washed,shein,shorts,0 1,denim


In [81]:
!mkdir -p web_dataset/train

In [82]:
from PIL import Image

In [83]:
df_products.dtypes

product_id       object
url              object
images           object
title            object
shop             object
category         object
category_code    object
material         object
dtype: object

In [87]:
products = []

IMAGES_PATH = os.path.join(CLOSE_DATASET_PATH, "images")
#Walk over the files in the dataset
print("Walking over the dataset of images", IMAGES_PATH)

for directory in os.listdir(IMAGES_PATH):
    for root, dirs, files in os.walk(os.path.join(IMAGES_PATH, directory)):
        product_info = df_products[df_products['product_id'] == str(directory)]
        product_title = product_info['title'].values[0].strip()
        if not product_title:
            continue
        for file in files:
            file_id = file.split(".")[0]
            web_image = Image.open(os.path.join(root, file))
            png_image = web_image.convert("RGBA")
            png_image.save(os.path.join("web_dataset/train",file_id + ".png"), "PNG")
            products.append({
                "file_name": file_id + ".png",
                'text': product_title
            })
            
        

Walking over the dataset of images /Users/valentinafeve/Datasets/13Apr/images


In [88]:
products

[{'file_name': '6243569467330711795.png',
  'text': 'vacation beach knot side solid mini skirt'},
 {'file_name': '2773059819033699560.png',
  'text': 'vacation beach knot side solid mini skirt'},
 {'file_name': '-7021926552904298585.png',
  'text': 'vacation beach knot side solid mini skirt'},
 {'file_name': '-1610666071621226324.png', 'text': 'hip lift leggings'},
 {'file_name': '2950010157878600578.png', 'text': 'hip lift leggings'},
 {'file_name': '5654109773805470484.png', 'text': 'hip lift leggings'},
 {'file_name': '-260367494211401579.png',
  'text': 'large sequin embellished slim fit mini skirt for parties'},
 {'file_name': '-8510239862214670492.png',
  'text': 'large sequin embellished slim fit mini skirt for parties'},
 {'file_name': '5654109773805470484.png',
  'text': 'large sequin embellished slim fit mini skirt for parties'},
 {'file_name': '-8430590023419562122.png',
  'text': 'large sequin embellished slim fit mini skirt for parties'},
 {'file_name': '-48845424085259947

In [91]:
import jsonlines
import json

with jsonlines.open('web_dataset/train/metadata.jsonl', mode='w') as writer:
    for product in products:
        writer.write(product)

In [76]:
import json
with open('web_dataset/train/metadata.jsonl', 'w+') as f:
    for product in products:
        if product['text'] == '':
            continue
        json.dump(product, f)
        f.write('\n')

In [92]:
#Load jsonl file with pandas
df_metadata = pd.read_json('web_dataset/train/metadata.jsonl', lines=True)
df_metadata.head()

,file_name,text
0,6243569467330711795.png,vacation beach knot side solid mini skirt
1,2773059819033699560.png,vacation beach knot side solid mini skirt
2,-7021926552904298585.png,vacation beach knot side solid mini skirt
3,-1610666071621226324.png,hip lift leggings
4,2950010157878600578.png,hip lift leggings


In [95]:
from datasets import load_dataset

dataset = load_dataset("web_dataset", data_dir="train")
dataset

Generating train split: 4943 examples [00:00, 37276.03 examples/s]


DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 4943
    })
})

In [ ]:
# Validate all images are loaded
for 

In [99]:
dataset.push_to_hub("valentinafeve/clothing_described")

Uploading the dataset shards: 100%|██████████| 4/4 [01:43<00:00, 25.79s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/valentinafeve/clothing_described/commit/53bac1033f7b3dcd64363b8f7f197255350e93db', commit_message='Upload dataset', commit_description='', oid='53bac1033f7b3dcd64363b8f7f197255350e93db', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
dataset = load_dataset("web_dataset", data_dir="train")